In [1]:
import pandas as pd
import requests
import zipfile
import os
import shutil
import time
from google.cloud import storage
from google.cloud.storage import Client, transfer_manager

In [ ]:
    url = "http://labrosa.ee.columbia.edu/~dpwe/tmp/millionsongsubset.tar.gz"
    
    output_path = "songs.tar.gz"
    print('Download starting...')
    req = requests.get(url, stream=True)
    
    # Get the total file size
    total_size = int(req.headers.get('content-length', 0))
    block_size = 1024  # 1 KB
    
    with open(output_path, 'wb') as f:
        for data in req.iter_content(block_size):
            f.write(data)
            # Calculate the download progress
            downloaded_size = f.tell()
            progress = int((downloaded_size / total_size) * 100)
            sys.stdout.write(f"\rProgress: {progress}%")
            sys.stdout.flush()
    
    print('\nDownload finished!')

In [3]:
url = "https://databank.worldbank.org/data/download/EdStats_CSV.zip"
output_path = "education_stats.zip"
req = requests.get(url)

with open(output_path, 'wb') as f:
    f.write(req.content)

In [4]:
zip_file = output_path
extracted_files = "stats"
with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall("stats")


In [5]:
# extract files from the zip folder
extracted_files = "stats"
contents = os.listdir(extracted_files)
files_to_upload = []
for item in contents:
    files_to_upload.append(item)
    print(item ," added to the array")


EdStatsCountry.csv  added to the array
EdStatsFootNote.csv  added to the array
EdStatsSeries.csv  added to the array
EdStatsCountry-Series.csv  added to the array
EdStatsData.csv  added to the array


In [ ]:
# delete zip folder
if os.path.exists(zip_file):
    os.unlink(zip_file)

In [10]:
bucket_name = 'promptmebebe'
source_directory = 'stats'
workers = 8

storage_client = Client()
bucket = storage_client.bucket(bucket_name)

results = transfer_manager.upload_many_from_filenames(
    bucket, files_to_upload, source_directory=source_directory, max_workers=workers
)
for name, result in zip(files_to_upload, results):
    # The results list is either `None` or an exception for each filename in
    # the input list, in order.
    if isinstance(result, Exception):
        print("Failed to upload {} due to exception: {}".format(name, result))
    else:
        print("Uploaded {} to {}.".format(name, bucket.name))

Uploaded EdStatsCountry.csv to promptmebebe.
Failed to upload EdStatsFootNote.csv due to exception: ('Connection aborted.', TimeoutError('The write operation timed out'))
Uploaded EdStatsSeries.csv to promptmebebe.
Uploaded EdStatsCountry-Series.csv to promptmebebe.
Failed to upload EdStatsData.csv due to exception: ('Connection aborted.', TimeoutError('The write operation timed out'))


In [19]:
# Upload the files to gcs bucket
def upload_files(bucket_name, source_directory_path, destination_directory_name):
    """Uploads multiple files to a given directory in a Google Cloud Storage bucket."""
    # Initialize a storage client.
    storage_client = storage.Client()

    # Get GCS bucket.
    bucket = storage_client.get_bucket(bucket_name)

    # List all files in the local directory.
    files = os.listdir(source_directory_path)

    file = 'EdStatsCountry.csv'
    file_path = os.path.join(source_directory_path, file)
    try:
        # Construct a blob.
        blob = bucket.blob(f'{destination_directory_name}/{file}')
        # Upload the local file to GCS.
        blob.upload_from_filename(file_path, timeout=500)
        
    except ConnectionError as e:
        time.sleep(delay)
        print(e)

bucket_name = 'promptmebebe'
destination_directory_name = 'education_stats'
file_path = 'stats'
upload_files(bucket_name, file_path, destination_directory_name)


Failed to upload file EdStatsCountry.csv


In [ ]:
# delete the data in the directory after uploading to gcs bucket
folder_path = 'stats'
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            for subfile in os.listdir(file_path):
                subfile_path = os.path.join(file_path, subfile)
                os.unlink(subfile_path)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

In [ ]:
if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

import requests
import os
import zipfile

def download(url, download_path):
    r = requests.get(url)
    
    with open(download, 'wb') as f:
        f.write(r.content)
def unzip(zip_file, unzipped_path):

    with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall(unzipped_path)

def clean_up(folder_path, zip_file):
    for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            for subfile in os.listdir(file_path):
                subfile_path = os.path.join(file_path, subfile)
                os.unlink(subfile_path)

        if os.path.isfile(zip_file) or os.path.islink(zip_file):
            os.unlink(zip_file)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")
    

@data_loader
def load_data(*args, **kwargs):
   url = 'url = "https://databank.worldbank.org/data/download/EdStats_CSV.zip"'
   download_path = 'ed_stats.zip'
   data_folder = 'stats'
   download(url, download_path)
   unzip(download_path, data_folder)
   file_name = 'EdStatsCountry.csv'
   df = pd.read_csv(f'{data_folder}/{file_name}')
   clean_up(data_folder, download_path)
    return df


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'


In [14]:
ed1 = 'stats/EdStatsCountry.csv' 
ed2 = 'stats/EdStatsFootNote.csv'
ed3 = 'stats/EdStatsSeries.csv'
ed4 = 'stats/EdStatsCountry-Series.csv' 
ed5 = 'stats/EdStatsData.csv'
df = pd.read_csv(ed1)
df.

Index(['Country Code', 'Short Name', 'Table Name', 'Long Name', '2-alpha code',
       'Currency Unit', 'Special Notes', 'Region', 'Income Group', 'WB-2 code',
       'National accounts base year', 'National accounts reference year',
       'SNA price valuation', 'Lending category', 'Other groups',
       'System of National Accounts', 'Alternative conversion factor',
       'PPP survey year', 'Balance of Payments Manual in use',
       'External debt Reporting status', 'System of trade',
       'Government Accounting concept', 'IMF data dissemination standard',
       'Latest population census', 'Latest household survey',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', 'Unnamed: 31'],
      dtype='object')

In [4]:
df = pd.read_csv(ed2)
df.head()

,CountryCode,SeriesCode,Year,DESCRIPTION,Unnamed: 4
0,ABW,SE.PRE.ENRL.FE,YR2001,Country estimation.,NaN
1,ABW,SE.TER.TCHR.FE,YR2005,Country estimation.,NaN
2,ABW,SE.PRE.TCHR.FE,YR2000,Country estimation.,NaN
3,ABW,SE.SEC.ENRL.GC,YR2004,Country estimation.,NaN
4,ABW,SE.PRE.TCHR,YR2006,Country estimation.,NaN


In [5]:
df = pd.read_csv(ed3)
df.head()

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,BAR.NOED.1519.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 15-19 with...,Percentage of female population age 15-19 with...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAR.NOED.1519.ZS,Attainment,Barro-Lee: Percentage of population age 15-19 ...,Percentage of population age 15-19 with no edu...,Percentage of population age 15-19 with no edu...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAR.NOED.15UP.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 15+ with n...,Percentage of female population age 15+ with n...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAR.NOED.15UP.ZS,Attainment,Barro-Lee: Percentage of population age 15+ wi...,Percentage of population age 15+ with no educa...,Percentage of population age 15+ with no educa...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BAR.NOED.2024.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 20-24 with...,Percentage of female population age 20-24 with...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = pd.read_csv(ed4)
df.head()

,CountryCode,SeriesCode,DESCRIPTION,Unnamed: 3
0,ABW,SP.POP.TOTL,Data sources : United Nations World Population...,NaN
1,ABW,SP.POP.GROW,Data sources: United Nations World Population ...,NaN
2,AFG,SP.POP.GROW,Data sources: United Nations World Population ...,NaN
3,AFG,NY.GDP.PCAP.PP.CD,Estimates are based on regression.,NaN
4,AFG,SP.POP.TOTL,Data sources : United Nations World Population...,NaN


In [13]:
df = pd.read_csv(ed5)
df[['2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2020']].head()

,2010,2011,2012,2013,2014,2015,2016,2017,2020
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85.211998,85.24514,86.101669,85.51194,85.320152,NaN,NaN,NaN,NaN
